In [12]:
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from sqlalchemy import create_engine
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
html = browser.html
soup = bs(html, 'html.parser')

In [15]:
def scrap_selections(countries, years):
    selection_players = []

    for country in countries:
        for year in years:
            current_page = f'https://www.national-football-teams.com/country/{countries[country]}/{year}/{country}.html'
            table = pd.read_html(current_page)[0]
            table.columns = table.columns.droplevel()
            table['National Team'] = country
            table['year'] = year
            selection_players.append(table)

    selection_players_df = pd.concat(selection_players)
    selection_players_df = selection_players_df.reset_index().drop(selection_players_df.
                                                                   columns[selection_players_df
                                                                           .columns.str.contains('unnamed', case=False)],
                                                                    axis=1).drop(['index'], axis=1)
    selection_players_df.columns = ['name', 'date_of_birth', 'position', 'current_club', 'f_matches', 'f_substitutions',
                                   'f_goals', 'nf_matches', 'nf_substitutions', 'nf_goals', 'national_team', 'year']
    selection_players_df.index.name = 'id'
    return selection_players_df

In [50]:
def csv_to_postgres(path, table_name, delimiter):
    df = pd.read_csv(path, sep=delimiter)
    connection_string = "postgres:postgres@localhost:5432/Fifa"
    engine = create_engine(f'postgresql://{connection_string}')
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

In [46]:
countries = {'Brazil':28, 'Mexico': 121, 'France': 67}
years = np.arange(2017, 2020)

players_df = scrap_selections(countries, years)
players_df.to_csv("fifa_selection_players.csv")

In [58]:
csv_to_postgres("fifa_ranking-2019-06-14.csv", "Countries", ';')

In [60]:
csv_to_postgres("fifa_selection_players.csv", "Players", ',')